## 数据挖掘实践任务
任务描述

- 用逻辑回归、svm和决策树、随机森林和XGBoost进行模型构建（不需考虑调参）

In [68]:
# 获取经过task02中特征选择处理过的数据集
import pandas as pd
dataset = pd.read_csv('./dataset/task2_proc.csv')
features = dataset.iloc[:, :-1]
labels = dataset.iloc[:, -1]
print('features shape:{}, labels shape:{}'.format(features.shape, labels.shape))

features shape:(4455, 50), labels shape:(4455,)


In [69]:
# 数据拆分
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


random_state=2018    # 后面会多次用到，故设置一个变量

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.3,
                                                    random_state=2018)

# 对训练集进行标准话处理
scaler = StandardScaler()
x_train_scale = scaler.fit_transform(x_train)
x_test_scale = scaler.fit_transform(x_test)

In [70]:
from sklearn.linear_model import LogisticRegression    # 逻辑回归 
from sklearn.svm import SVC    # SVM 支持向量机
from sklearn.tree import DecisionTreeClassifier   # 决策树
from sklearn.ensemble import RandomForestClassifier   # 随机森林分类器
from sklearn.model_selection import RandomizedSearchCV, cross_val_predict  #  随机搜索，交叉验证
#　https://www.cnblogs.com/jiaxin359/p/8552800.html
from scipy.stats import uniform    # 标准均匀分布
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

In [71]:
model_score_train = []
decision_score_train = []
model_score_test = []
decision_score_test = []

def proc_score(y_pred, y_pred_scores, y_test, train=True):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    
    if train:
        decision_score_train.append(y_pred_scores)
        model_score_train.append([accuracy, precision, recall, roc_auc])
        text = 'Train'
    else:
        decision_score_test.append(y_pred_scores)
        model_score_test.append([accuracy, precision, recall, roc_auc])
        text = 'Test'
    
    print('{}: accuracy_score:{:.3f}, precision_score:{:.3f}, recall_score:{:.3f}, f1_score:{:.3f}, roc_auc_score:{:.3f}'.format(
    text, accuracy, precision, recall, f1, roc_auc))

## 逻辑回归

In [72]:
param_distributions = {'C':uniform(0,10)}
rs_clf  = RandomizedSearchCV(LogisticRegression(solver='liblinear'), 
                             param_distributions=param_distributions, 
                             cv=3,    # 
                             n_iter=100,
                             random_state=random_state,
                             verbose=2, # verbose是控制日志输出，0无输出，1简化输出，2详细输出
                             n_jobs=-1
                            )
rs_clf.fit(x_train_scale, y_train)
logistic = rs_clf.best_estimator_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    3.6s finished


In [73]:
y_pred = logistic.predict(x_train_scale)
y_pred_scores = cross_val_predict(logistic, 
                                  x_train_scale, 
                                  y_train, 
                                  cv=5,    # integer, to specify the number of folds in a (Stratified)KFold,
                                  method='decision_function'
                                  # method : string, optional, default: ‘predict’
                                 )
# 使用sklearn进行模型交叉验证：  https://www.cnblogs.com/jiaxin359/p/8552800.html
proc_score(y_pred, y_pred_scores, y_train, train=True)

y_pred = logistic.predict(x_test_scale)
y_pred_scores = cross_val_predict(logistic, x_test_scale, y_test, cv=5,
                                  method='decision_function')
proc_score(y_pred, y_pred_scores, y_test, train=False)

Train: accuracy_score:0.799, precision_score:0.713, recall_score:0.355, f1_score:0.474, roc_auc_score:0.653
Test: accuracy_score:0.791, precision_score:0.624, recall_score:0.349, f1_score:0.448, roc_auc_score:0.641


## SVM

In [ ]:
param_distributions = {'C':uniform(0,10)}
rs_clf = RandomizedSearchCV(SVC(kernel='linear'),
                            param_distributions=param_distributions,
                            cv=3,
                            n_iter=100,
                            random_state=random_state,
                            verbose=2,
                            n_jobs=-1
                           )
rs_clf.fit(x_train_scale, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.4s


In [ ]:
svc = rs_clf.best_estimator_

y_pred = svc.predict(x_train_scale)
y_pred_scores = cross_val_predict(svc, x_train_scale, y_train,cv=3,
                                 method='decision_function')
proc_score(y_pred, y_pred_scores, y_train, train=True)

y_pred = svc.predict(x_test_scale)
y_pred_scores = cross_val_predict(svc, x_test_scale, y_test, cv=3,
                                 method='decision_function')
proc_score(y_pred, y_pred_scores, y_test, train=False)

## 决策树

In [ ]:
dt_clf = DecisionTreeClassifier(max_depth=8)
dt_clf.fit(x_train_scale, y_train)

In [ ]:

y_pred = dt_clf.predict(x_train_scale)
y_pred_scores = cross_val_predict(dt_clf,x_train_scale,y_train,cv=5,
                                method='predict_proba')
proc_score(y_pred,y_pred_scores[:,1],y_train,train=True)

y_pred = dt_clf.predict(x_test_scale)
y_pred_scores = cross_val_predict(dt_clf,x_test_scale,y_test,cv=5,
                                method='predict_proba')
proc_score(y_pred,y_pred_scores[:,1],y_test,train=False)

## 随机森林

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=1000,  # 也就是弱学习器的最大迭代次数，或者说最大的弱学习器的个数
                                criterion='gini',  # criterion: 即CART树做划分时对特征的评价标准
                                oob_score=True,  # oob_score :即是否采用袋外样本来评估模型的好坏
                                random_state=random_state,
                                verbose=0, 
                                n_jobs=-1
                               )
rf_clf.fit(x_train_scale, y_train)
rf_clf.oob_score_

In [ ]:
y_pred = rf_clf.predict(x_train_scale)
y_pred_scores = cross_val_predict(rf_clf,x_train_scale,y_train,cv=5,
                                method='predict_proba')
proc_score(y_pred,y_pred_scores[:,1],y_train,train=True)

y_pred = rf_clf.predict(x_test_scale)
y_pred_scores = cross_val_predict(rf_clf,x_test_scale,y_test,cv=5,
                                method='predict_proba')
proc_score(y_pred,y_pred_scores[:,1],y_test,train=False)

## XGBoost

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(x_train_scale, y_train)

In [ ]:
y_pred = xgb.predict(x_train_scale)
y_pred_scores = cross_val_predict(xgb,x_train_scale,y_train,cv=5,
                                method='predict_proba')
proc_score(y_pred,y_pred_scores[:,1],y_train,train=True)

y_pred = xgb.predict(x_test_scale)
y_pred_scores = cross_val_predict(xgb,x_test_scale,y_test,cv=5,
                                method='predict_proba')
proc_score(y_pred,y_pred_scores[:,1],y_test,train=False)

## 对个模型进行比较

In [ ]:
model_name = ['Logistic', 'SVM', 'DecisionTree', 'RandomForest', 'XGBoost']
columns = ['accuracy_score', 'precision_score', 'recall_score', 'f1_score', 'roc_auc_score']
types = ['train', 'test']
pd_list = []
for train,test in zip(model_score_train,model_score_test):
    pd_list.append(pd.DataFrame([train,test],index=types,columns=columns))
    
pd.concat(pd_list,axis=0,keys=model_name) 

In [ ]:

# 绘制ROC取下对比图
def plot_roc_curve(fpr,tpr,label=None):
    plt.plot(fpr,tpr,linewidth=2,label=label)
    plt.plot([0,1],[0,1],'k--')

for score,label in zip(decision_score_test,['Logistic','SVM','DecisionTree','RandomForest','xgboost']):
    fpr,tpr,threshold = roc_curve(y_test,score)
    plot_roc_curve(fpr,tpr,label)
plt.rcParams['figure.figsize'] = (8,5)    
plt.axis([0,1,0,1])
plt.xlabel('False Position Rate')
plt.ylabel('False Position Rate')
plt.legend()
plt.show()